<br>
<br>

# **Modelos del lenguaje basados en redes neuronales artificiales**

## **Redes neuronales recurrentes (RNN)**

### **LSTM**

El conjunto de datos "AG_NEWS" es un conjunto de datos de clasificación de texto ampliamente utilizado en el campo del procesamiento de lenguaje natural (NLP). Contiene noticias de diferentes categorías y se utiliza comúnmente para tareas de clasificación de texto. El conjunto de datos AG_NEWS consta de noticias de cuatro categorías principales, que son:

1. **World**: Noticias sobre eventos y acontecimientos globales, como política internacional, relaciones internacionales y noticias mundiales en general.

2. **Sports**: Noticias relacionadas con eventos deportivos, resultados de partidos, eventos deportivos nacionales e internacionales, etc.

3. **Business**: Noticias relacionadas con el mundo de los negocios, finanzas, economía, empresas, informes de ganancias y otros temas económicos.

4. **Sci/Tech**: Noticias relacionadas con ciencia y tecnología, incluyendo avances científicos, novedades tecnológicas, gadgets, investigaciones científicas y más.

Cada instancia del conjunto de datos AG_NEWS generalmente consiste en un título y un cuerpo de una noticia, junto con una etiqueta que indica la categoría a la que pertenece. 

In [12]:
from torchtext import datasets
from torchtext.data import to_map_style_dataset
import numpy as np

# Load the dataset
train_iter, test_iter = datasets.AG_NEWS(split=('train', 'test'))

train_ds = to_map_style_dataset(train_iter)
test_ds = to_map_style_dataset(test_iter)

train = np.array(train_ds)
test = np.array(test_ds)

In [13]:
# Create vocabulary and embedding

from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer("basic_english")

vocab = build_vocab_from_iterator(map(lambda x: tokenizer(x[1]), train_iter), specials=['<pad>','<unk>'])
vocab.set_default_index(vocab["<unk>"])

In [14]:
print("Tamaño del vocabulario:", len(vocab), "tokens")
print("Tokenización de la frase 'Here is an example sentence':", tokenizer("Here is an example sentence"))
print("Índices de las palabras 'here', 'is', 'an', 'example', 'supercalifragilisticexpialidocious':", vocab(['here', 'is', 'an', 'example', 'supercalifragilisticexpialidocious']))
print("Palabras correspondientes a los índices 475, 21, 30, 5297, 0:", vocab.lookup_tokens([475, 21, 30, 5297, 0]))
print("Las diez primeras palabras del vocabulario:", vocab.get_itos()[:10])

Tamaño del vocabulario: 95812 tokens
Tokenización de la frase 'Here is an example sentence': ['here', 'is', 'an', 'example', 'sentence']
Índices de las palabras 'here', 'is', 'an', 'example', 'supercalifragilisticexpialidocious': [476, 22, 31, 5298, 1]
Palabras correspondientes a los índices 475, 21, 30, 5297, 0: ['version', 'at', 'from', 'establish', '<pad>']
Las diez primeras palabras del vocabulario: ['<pad>', '<unk>', '.', 'the', ',', 'to', 'a', 'of', 'in', 'and']


In [15]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

print("Tokenización de la frase 'Here is an example sentence':", text_pipeline("Here is an example sentence"))

Tokenización de la frase 'Here is an example sentence': [476, 22, 31, 5298, 2994]


In [16]:
from torch.utils.data import DataLoader
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list = [], []
    for sample in batch:
        label, text = sample
        text_list.append(torch.tensor(text_pipeline(text), dtype=torch.long))
        label_list.append(label_pipeline(label))

        
    return torch.tensor(label_list, dtype=torch.long), torch.nn.utils.rnn.pad_sequence(text_list, batch_first=True, padding_value=vocab["<pad>"])

train_dataloader = DataLoader(
    train_iter, batch_size=64, shuffle=True, collate_fn=collate_batch
)

test_dataloader = DataLoader(
    test_iter, batch_size=64, shuffle=True, collate_fn=collate_batch
)

Para verificar que estamos creando correctamente los lotes, vamos a imprimir las primeras cuatro instancias:

In [17]:
for batch in train_dataloader:
    print(batch[1][:4])
    print("\n")
    print(batch[0][:4])
    print("\n")
    break

tensor([[  152,    44,    11,  1618,   483,   343,   149,    24,    74,    14,
            32,    15,    16,     6,   669,     8,   483,    93,     9,     6,
          1194,     8,   453,    59,  1242,    58,  1878,   361,     5,   431,
          2777,  1272,    69,  1417,     4,    21,   202,    12,     3,   950,
          1315,     4,     9,    68,  1015,   152,  6540,   281,     2,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [ 6090,  1510, 10335,   414,    12, 41858,   237,    14,    32,    15,
           189,   191,  6090,  1696,  2795,    23,    48, 27190,     8,     3,
            37,   483,  1829,   127,    57,     4,   604,    26,    96,   275,
         16502,    55, 10335,   414,    88,     2,    30,   365,  8933,   191,
             7,  2240,    12

In [18]:
import torch
import torch.nn as nn

class LSTMTextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super(LSTMTextClassificationModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_class)

    def forward(self, text):
        embedded = self.embedding(text)
        lstm_out, _ = self.lstm(embedded)
        # Tomar la última salida de la secuencia LSTM
        last_output = lstm_out[:, -1, :]
        output = self.fc(last_output)
        return output


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LSTMTextClassificationModel(len(vocab), 32, 64, 4).to(device)
model.train()

for batch in train_dataloader:
    predicted_label = model(batch[1])
    label = batch[0]
    break

print(batch[1][:4])
print(predicted_label[:4])
print(label[:4])

tensor([[  203,    85,  1644,  1605,    29,  1480,    24,    74,    14,    28,
            15,    16,   203,    65,     2,    27,    11,    57,    18,     3,
          1605,    29,  1480,    12,    23,  1385,   385,   727,     2,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  754, 19257, 18337,    39,    17,  2675,   136, 13709,    17,   352,
            14,    28,    15,    16,     8,    62,     7,     3,   144,  8149,
         21035,     5, 10132,    21,     3,   352, 

In [20]:
import time

# Hyperparameters
EPOCHS = 10  # epoch
LR = 5  # learning rate

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

def train(dataloader):
    model.train()
    total_acc, total_count, max_acc = 0, 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()

        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            
            #print(total_acc / total_count)

            print('| {:5d} batches '
                  '| accuracy {:8.3f}'.format(idx, total_acc / total_count))

            if max_acc < total_acc / total_count:
                max_acc = total_acc / total_count
                
            total_acc, total_count = 0, 0
            start_time = time.time()
    return max_acc


def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            predicted_label = model(text)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

In [21]:
# Hyperparameters
EPOCHS = 10  # epoch
LR = 5  # learning rate
BATCH_SIZE = 64  # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)


for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()

    accu_train = train(train_dataloader)
    accu_val = evaluate(test_dataloader)

    #if accu_train > accu_val:
    #    scheduler.step()
    
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print("-" * 59)

|   500 batches | accuracy    0.254
|  1000 batches | accuracy    0.256
|  1500 batches | accuracy    0.257
-----------------------------------------------------------
| end of epoch   1 | time: 59.81s | valid accuracy    0.252 
-----------------------------------------------------------
|   500 batches | accuracy    0.259
|  1000 batches | accuracy    0.256
|  1500 batches | accuracy    0.255
-----------------------------------------------------------
| end of epoch   2 | time: 60.92s | valid accuracy    0.253 
-----------------------------------------------------------
|   500 batches | accuracy    0.352
|  1000 batches | accuracy    0.464
|  1500 batches | accuracy    0.532
-----------------------------------------------------------
| end of epoch   3 | time: 61.16s | valid accuracy    0.567 
-----------------------------------------------------------
|   500 batches | accuracy    0.588
|  1000 batches | accuracy    0.671
|  1500 batches | accuracy    0.741
-------------------------

---

### Ejercicio 1

Modifica el código anterior para adaptar el modelo LSTM al uso de *embeddings* preentrenados. Para ello, usa <code>from torchtext.vocab import GloVe</code> y elige el conjunto de *embeddings* GloVe que prefieras. Puedes encontrar más información en https://pytorch.org/text/stable/vocab.html#torchtext.vocab.GloVe


---